In [1]:
!pip install Levenshtein

     ---------------------------------------- 98.3/98.3 kB 2.8 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
from storage import MongoDBStorage
from service import MongoDBSearchService
import numpy as np
import Levenshtein

In [41]:
with open("db_host", "r") as file:
    host = file.read().rstrip()

storage = MongoDBStorage(parameters={
        "host_url": host
    })
search_service = MongoDBSearchService(storage)

In [47]:
# calculates similarity using levenshtein distance,
# returns 2d array of x_keywords compared to y_keywords
def calculate_similarity(x_keywords, y_keywords):
    data = []
    for x_label in x_keywords:
        data_part = []
        data.append(data_part)
        for y_label in y_keywords:
            data_part.append(Levenshtein.ratio(x_label, y_label))
    return data

In [48]:
cursor = search_service.aggregate_filters_with_or([
        search_service.get_query_find_by_id('661931f5d172007aa8b18204'),
        search_service.get_query_find_by_id('661931fcd172007aa8b18205'),
        search_service.get_query_find_by_id('661931ffd172007aa8b18206'),
        search_service.get_query_find_by_id('663f15afd9798a05aae49f74'),
        search_service.get_query_find_by_id('663f15b0d9798a05aae49f75'),
        search_service.get_query_find_by_id('663f15b0d9798a05aae49f76')
    ])
footprints_to_link = []
for test_df in cursor:
    footprints_to_link.append(test_df)


In [51]:
def get_mean_val(footprint_l, footprint_r, label):
    data = calculate_similarity(footprint_l[label], footprint_r[label])
    return np.array(data).mean()

def add_connection(footprint, object_id, score, linked_by_label):
    if 'associated_objects' not in footprint:
        footprint['associated_objects'] = []
    has_same_object = False
    for associated_object in footprint['associated_objects']:
        if associated_object['id'] == object_id:
            has_same_object = True
            break

    if not has_same_object:
        footprint['associated_objects'].append({
            "id": object_id,
            "linked_by_label": linked_by_label,
            "score": score
        })

connection_labels = ['keywords', 'topics']
# connecting footprint between themselves
for footprint in footprints_to_link:
    for footprint_inner in footprints_to_link:
        if footprint['_id'] == footprint_inner['_id']:
            continue

        for label in connection_labels:
            # connecting by keywords
            mean_val = get_mean_val(footprint, footprint_inner, label)
            if mean_val > 0.5:
                add_connection(footprint, str(footprint_inner['_id']), mean_val, label)
                add_connection(footprint_inner, str(footprint['_id']), mean_val, label)
                continue

In [53]:
scores = []
for footprint in footprints_to_link:
    if 'associated_objects' in footprint:
        for object in footprint['associated_objects']:
            scores.append(object['score'])

print(f"Mean score for connections is {np.array(scores).mean()}")

Mean score for connections is 0.6432446577589314


In [46]:
for df in footprints_to_link:
    if '_id' in df:
        query = search_service.get_query_find_by_id(id=df['_id'])
        storage.update_df_raw(query, df)